In [1]:
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from nltk.corpus import stopwords
import re
import string
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import NearestNeighbors

In [2]:
client_data = pd.read_csv('./data/clients.csv')
student_data = pd.read_csv('./data/fyp_students_datasets.csv')

In [3]:
client_data.head(3)

,Company Name,Title,Project brief,Project Background,Requiements
0,ATX,ATX Software Testing Suite,Automated testing suite for all software produ...,This project will focus on graphical user inte...,Generates user interface events and observes t...
1,NaN,Prediction on Obesity associates with Lifestyl...,Identify the effective indicators of obesity b...,The main objective of the present study is to ...,To propose the features in determining obesity...
2,NaN,Mobile Advertising Platform with Designing Sui...,To improve employee communications and engagem...,The goal of the project is to create a mobile ...,to provide direct communications on employment...


In [4]:
student_data.head(4)

,Group No.,Unnamed: 1,Title,Supervsior,Met with supervisor?,Preferred name on the CASUGOL certificate,FULL NAME As your IC/Passport,CV sent\n(Y/N),Program,Areas of Interest and your technical strength,...,Leader / Member,Email ADD,SD EMAIL,Contact No,Semester,Check Study Plan: Capstone 2 is it JAN-FEB,CGPA,GPA,Industry - Contact Point,Company
0,1.0,1.0,A Trust Index Model to Detect Fake News in Soc...,Mr. Tee Wee Jing,Yes,Nicholas Ryan Farm,Nicholas Ryan Farm,Yes,Bachelor of Computer Science,"Full Stack Web Development, Project Management...",...,Leader,nicholasfarm27@gmail.com,nicholasryan.farm@sd.taylors.edu.my,019-4549277,Sem 4,Yes,3.94,3.50,NaN,ProximaX
1,NaN,2.0,NaN,NaN,NaN,Gan Zheng Jie,Gan Zheng Jie,Yes,Bachelor of Software Engineering,"Full stack mobile application development, Dat...",...,Member,ganzj45@gmail.com,zhengjie.gan@sd.taylors.edu.my,017-5015966,Sem 5,Yes,3.85,3.78,NaN,NaN
2,NaN,3.0,NaN,NaN,NaN,Low Yee Yieng,Low Yee Yieng,Yes,Bachelor of Software Engineering,"IT Analysis, Web/Mobile App Development, UI/UX...",...,Member,lyeeyieng@gmail.com,yeeyieng.low@sd.taylors.edu.my,010-2776323,Sem 5,Yes,3.87,3.75,NaN,NaN
3,NaN,4.0,NaN,NaN,NaN,Sachin Vijaya Kumar,Saachin Vijaya Kumar,Yes,Bachelor of Computer Science,"Front-end development, UI/UX Design, Data Anal...",...,Member,taylorssvk@gmail.com,saachin.vijayakumar@sd.taylors.edu.my,018-3137551,Sem 5,Yes,3.59,3.67,NaN,NaN


# Exploratory Data Analysis

In [5]:
client_data['Requirements'] = client_data['Project brief '].map(str) + ' ' + client_data['Project Background']+' '+client_data['Requiements']

In [6]:
client_data.head(3)


,Company Name,Title,Project brief,Project Background,Requiements,Requirements
0,ATX,ATX Software Testing Suite,Automated testing suite for all software produ...,This project will focus on graphical user inte...,Generates user interface events and observes t...,Automated testing suite for all software produ...
1,NaN,Prediction on Obesity associates with Lifestyl...,Identify the effective indicators of obesity b...,The main objective of the present study is to ...,To propose the features in determining obesity...,Identify the effective indicators of obesity b...
2,NaN,Mobile Advertising Platform with Designing Sui...,To improve employee communications and engagem...,The goal of the project is to create a mobile ...,to provide direct communications on employment...,To improve employee communications and engagem...


In [7]:
client_data = client_data.drop(['Requiements'],axis=1)

In [8]:
client_data = client_data.fillna(' ')

In [9]:
client_data.head()

,Company Name,Title,Project brief,Project Background,Requirements
0,ATX,ATX Software Testing Suite,Automated testing suite for all software produ...,This project will focus on graphical user inte...,Automated testing suite for all software produ...
1,,Prediction on Obesity associates with Lifestyl...,Identify the effective indicators of obesity b...,The main objective of the present study is to ...,Identify the effective indicators of obesity b...
2,,Mobile Advertising Platform with Designing Sui...,To improve employee communications and engagem...,The goal of the project is to create a mobile ...,To improve employee communications and engagem...
3,PurposePurchase Sdh Bhd,PurposePurchase,"\ncause-related ecommerce app, as well as a pl...",PurposePurchase is an online e-commerce platfo...,"\ncause-related ecommerce app, as well as a pl..."
4,,Educational Game,The aim of this project is to create an educat...,"The students will be given a concept document,...",The aim of this project is to create an educat...


In [10]:
client_data.shape

(25, 5)

In [11]:
stopwords_ = stopwords.words('english')
stopword_ = set(stopwords.words('english'))
wn = WordNetLemmatizer()

In [12]:
# Create word tokens
def token_txt(token):
    return token not in stopword_ and token not in list(string.punctuation) and len(token) > 2   
  

In [13]:
def clean_txt(text):
  clean_text = []
  clean_text2 = []
  text = re.sub("'", "", text)
  text = re.sub("(\\d|\\W)+", " ", text) 
  text = text.replace("nbsp", "")
  clean_text = [wn.lemmatize(word, pos = "v") for word in word_tokenize(text.lower()) if token_txt(word)]
  clean_text2 = [word for word in clean_text if token_txt(word)]
  return " ".join(clean_text2)

In [14]:
client_data['Requirements'] = client_data['Requirements'].apply(clean_txt)

In [15]:
client_data.tail()

,Company Name,Title,Project brief,Project Background,Requirements
20,,,,,
21,Agensi Pekerjaan & Latihan TAPRO Sdn Bhd,(CRM) tool for recruitment.,"Currently, selected information of the candida...",Aim to build a CRM tool for recruitment,currently select information candidates extrac...
22,People Systems Consultancy Sdn Bhd,App to build Financial Inclusion and Intellige...,Build the financial resilience and intelligenc...,seeking support and assistance to: \n(a) build...,build financial resilience intelligence differ...
23,,Statistics: Zero to Hundred,"In the world of education, e-learning platform...",The aim of this project is to create an e-lear...,world education learn platform essential make ...
24,,WhatsApp Messenger Forensic Analysis Based on ...,,,


## Student corpus

In [16]:
student_data['Requirements'] = student_data['Program '].map(str) + ' ' + student_data['Areas of Interest and your technical strength']

In [17]:
student_data = student_data[['FULL NAME As your IC/Passport','Requirements']]

In [18]:
student_data.head(3)

,FULL NAME As your IC/Passport,Requirements
0,Nicholas Ryan Farm,Bachelor of Computer Science Full Stack Web De...
1,Gan Zheng Jie,Bachelor of Software Engineering Full stack m...
2,Low Yee Yieng,"Bachelor of Software Engineering IT Analysis,..."


In [19]:
student_data['Requirements'] = student_data['Requirements'].apply(clean_txt)

# Term Ferquency - Inverse Document Frequency

In [20]:
tfidf_vect = TfidfVectorizer()

# Fitting and transforming the vector
tfidf_comb = tfidf_vect.fit_transform((client_data['Requirements'])) 
tfidf_comb

<25x626 sparse matrix of type '<class 'numpy.float64'>'
	with 921 stored elements in Compressed Sparse Row format>

In [21]:
user = "Nicholas Ryan Farm"
index2 = np.where(student_data['FULL NAME As your IC/Passport'] == user)[0][0]

In [22]:
student_data_1 = student_data.iloc[[index2]]
student_data_1

,FULL NAME As your IC/Passport,Requirements
0,Nicholas Ryan Farm,bachelor computer science full stack web devel...


In [23]:
stud_rec = tfidf_vect.transform(student_data_1['Requirements'])
cos_stud_rec = list(map(lambda x: cosine_similarity(stud_rec, x), tfidf_comb))
cos_stud_rec

[array([[0.27707994]]),
 array([[0.]]),
 array([[0.01940262]]),
 array([[0.]]),
 array([[0.17834359]]),
 array([[0.04756199]]),
 array([[0.1065304]]),
 array([[0.]]),
 array([[0.16944361]]),
 array([[0.07722981]]),
 array([[0.04245635]]),
 array([[0.]]),
 array([[0.]]),
 array([[0.]]),
 array([[0.]]),
 array([[0.]]),
 array([[0.09570462]]),
 array([[0.15236234]]),
 array([[0.01794798]]),
 array([[0.]]),
 array([[0.]]),
 array([[0.0357513]]),
 array([[0.02905651]]),
 array([[0.0247534]]),
 array([[0.]])]

In [24]:
def get_recommendation(top, client_data, scores):
  recommendation = pd.DataFrame(columns = ['Student', 'Title',  'Company Name', 'Score'])
  count = 0
  for i in top:
      recommendation.at[count, 'Student'] = user
      recommendation.at[count, 'Title'] = client_data['Title'][i]
      recommendation.at[count, 'Company Name'] = client_data['Company Name'][i]
      recommendation.at[count, 'Score'] =  scores[count]
      count += 1
  return recommendation

In [25]:
top10_stud_rec = sorted(range(len(cos_stud_rec)), key = lambda i: cos_stud_rec[i], reverse = True)[:10]
top10_stud_rec 

[0, 4, 8, 17, 6, 16, 9, 5, 10, 21]

In [26]:
list_scores = [cos_stud_rec[i][0][0] for i in top10_stud_rec]
list_scores

[0.2770799408184324,
 0.178343594333634,
 0.16944361027034827,
 0.15236234238839746,
 0.10653040262232864,
 0.09570461863347014,
 0.07722980676345359,
 0.04756198608485304,
 0.04245634537212887,
 0.035751302364328975]

In [27]:
get_recommendation(top10_stud_rec, client_data, list_scores)

,Student,Title,Company Name,Score
0,Nicholas Ryan Farm,ATX Software Testing Suite,ATX,0.27708
1,Nicholas Ryan Farm,Educational Game,,0.178344
2,Nicholas Ryan Farm,\nNodejs Port of ATX Wallet System\n,ATX / MSC,0.169444
3,Nicholas Ryan Farm,Cyber Threat Intelligence Mining Model,,0.152362
4,Nicholas Ryan Farm,Content Management System in an IOT environment,Coinnov8 Web Portal,0.10653
5,Nicholas Ryan Farm,Hilti Global Territory Management Solution,Hilti,0.095705
6,Nicholas Ryan Farm,Nodejs Port of Mypospay Backend,ATX / MSC,0.07723
7,Nicholas Ryan Farm,Fake News Detection,ProximaX,0.047562
8,Nicholas Ryan Farm,Online Assessment in Education System,,0.042456
9,Nicholas Ryan Farm,(CRM) tool for recruitment.,Agensi Pekerjaan & Latihan TAPRO Sdn Bhd,0.035751


#  Top Recommendations with Count Vectorizer

In [28]:
count_vect = CountVectorizer()

# Fitting and transforming the vectorizer
count_comb = count_vect.fit_transform((client_data['Requirements'])) #fitting and transforming the vector
count_comb

<25x626 sparse matrix of type '<class 'numpy.int64'>'
	with 921 stored elements in Compressed Sparse Row format>

In [29]:
student_data_1_cv = count_vect.transform(student_data_1['Requirements'])
cv_cos_stud_data = list(map(lambda x: cosine_similarity(student_data_1_cv, x), count_comb))

In [31]:
top10_cv_count = sorted(range(len(cv_cos_stud_data)), key = lambda i: cv_cos_stud_data[i], reverse = True)[:10]
list_cv_count = [cv_cos_stud_data[i][0][0] for i in top10_cv_count]
get_recommendation(top10_cv_count, client_data, list_cv_count)

,Student,Title,Company Name,Score
0,Nicholas Ryan Farm,ATX Software Testing Suite,ATX,0.318198
1,Nicholas Ryan Farm,\nNodejs Port of ATX Wallet System\n,ATX / MSC,0.244451
2,Nicholas Ryan Farm,Educational Game,,0.21347
3,Nicholas Ryan Farm,Cyber Threat Intelligence Mining Model,,0.168364
4,Nicholas Ryan Farm,Nodejs Port of Mypospay Backend,ATX / MSC,0.11767
5,Nicholas Ryan Farm,Content Management System in an IOT environment,Coinnov8 Web Portal,0.116775
6,Nicholas Ryan Farm,Hilti Global Territory Management Solution,Hilti,0.103142
7,Nicholas Ryan Farm,Fake News Detection,ProximaX,0.08165
8,Nicholas Ryan Farm,Online Assessment in Education System,,0.055048
9,Nicholas Ryan Farm,Statistics: Zero to Hundred,,0.04461


In [34]:
print(student_data_1['Requirements'])

0    bachelor computer science full stack web devel...
Name: Requirements, dtype: object


# Top Recommendations using KNN

In [36]:
n_neighbors = 11
KNN = NearestNeighbors(n_neighbors = n_neighbors, p = 2)
KNN.fit(tfidf_comb)

NearestNeighbors(n_neighbors=11)

In [38]:
knn_rec = KNN.kneighbors(stud_rec, return_distance = True) 

In [40]:
knn_rec[0][0][1:]

array([1.        , 1.        , 1.        , 1.        , 1.        ,
       1.20243092, 1.28191763, 1.28884164, 1.30202739, 1.33676445])

In [41]:
top10_rec_knn = knn_rec[1][0][1:]
index_score_knn = knn_rec[0][0][1:]

get_recommendation(top10_rec_knn, client_data, index_score_knn)

,Student,Title,Company Name,Score
0,Nicholas Ryan Farm,,,1.0
1,Nicholas Ryan Farm,,,1.0
2,Nicholas Ryan Farm,,,1.0
3,Nicholas Ryan Farm,,,1.0
4,Nicholas Ryan Farm,WhatsApp Messenger Forensic Analysis Based on ...,,1.0
5,Nicholas Ryan Farm,ATX Software Testing Suite,ATX,1.202431
6,Nicholas Ryan Farm,Educational Game,,1.281918
7,Nicholas Ryan Farm,\nNodejs Port of ATX Wallet System\n,ATX / MSC,1.288842
8,Nicholas Ryan Farm,Cyber Threat Intelligence Mining Model,,1.302027
9,Nicholas Ryan Farm,Content Management System in an IOT environment,Coinnov8 Web Portal,1.336764
